In [1]:
!pip install yfinance
!pip install pytickersymbols

     |████████████████████████████████| 5.5MB 6.2MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=fd9f5ecbc3828ddf82089ac2486b62c0b7a3c79c62f729ee3ed8974832d2b863
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 81kB 3.4MB/s 
     |████████████████████████████████| 276kB 8.0MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=8dbf459de0170751eb883aba3a3bba77b94d72d83dba6b433876c0254600e473
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import yfinance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pytickersymbols import PyTickerSymbols
from google.colab import files
from datetime import datetime, timedelta
from tqdm.notebook import tqdm

In [3]:
sp400_url="https://en.wikipedia.org/wiki/List_of_S%26P_400_companies"
sp400, *_=pd.read_html(sp400_url)
sp400 = sp400['Ticker symbol'].to_list()

stock_data = PyTickerSymbols()

nasdaq_100=pd.DataFrame(stock_data.get_stocks_by_index('NASDAQ 100'))
nasdaq_100 = nasdaq_100[nasdaq_100.symbol != 'CTRP'].reset_index().symbol.to_list()

sp_500=pd.DataFrame(stock_data.get_stocks_by_index('S&P 500'))
sp_500 = sp_500[sp_500.symbol != 'SSE']
sp_500 = sp_500[sp_500.symbol != 'RTN']
sp_500 = sp_500[sp_500.symbol != 'CXO']
sp_500 = sp_500[sp_500.symbol != 'JEC']
sp_500 = sp_500[sp_500.symbol != 'SPY']
sp_500 = sp_500[sp_500.symbol != 'TIF'].reset_index().symbol.to_list()

stocks = nasdaq_100 + sp_500 #+ sp400
#stocks = ['AAPL']

stocks = set(stocks) #Convierte la lista a Set, que lo único que hace es quitar elementos repetidos
stocks = list(stocks) #Reconvertir el set a list (para poder iterar sobre ella)

In [4]:
len(stocks)

514

In [5]:
tol = 0
porcentaje = .015

In [6]:
#Obtener datos del día previo
# for i in tqdm(range(len(stocks))):
#   df = yfinance.download(stocks[i], period='2d')[['Low', 'High']]
#   data_day


In [16]:
igniting_long = []
igniting_short = []
for i in tqdm(range(len(stocks))):
  data = pd.DataFrame.copy(yfinance.download(stocks[i], period='2d', interval='5m')[['Low', 'High', 'Open', 'Close']])
  data_day = yfinance.download(stocks[i], period='2d')[['Low', 'High']]

  #gap: para saber si abrió hacia abajo o arriba
  gap = data.iloc[-2]['Open'] - data.iloc[-3]['Close'] 

  if gap >= 0:
    long_bar_percentage = (data.iloc[-2]['High'] - data.iloc[-2]['Low'])/data.iloc[-2]['Low']

    if (long_bar_percentage >= porcentaje and data['Close'][-2] > data['Open'][-2] 
        and data['Close'][-2] > data_day['High'].loc[data.index[-3].strftime('%Y-%m-%d')]):
      
      igniting_long.append(stocks[i])

  elif gap < 0:
    long_bar_percentage = (data.iloc[-2]['Low'] - data.iloc[-2]['High'])/data.iloc[-2]['High']

    if (long_bar_percentage < porcentaje and data['Open'][-2] > data['Close'][-2] 
        and data['Close'][-2] < data_day['Low'].loc[data.index[-3].strftime('%Y-%m-%d')]):
      
      igniting_short.append(stocks[i])

  #print(stocks[i])
  #print(data[-1:])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

KeyboardInterrupt: ignored

In [ ]:
igniting_long

In [ ]:
igniting_short

In [13]:
trigger_long = []
trigger_short = []

ign_size = data['High'][-2] - data['Low'][-2]

for i in range(len(igniting_long)):
  data = pd.DataFrame.copy(yfinance.download(igniting_long[i], period='2d', interval='5m')[['Low', 'High']])
  #Condiciones para Trigger: que el high de la segunda barra este por debajo del high Igniting Y que el Low segunda esté por arriba de la mitad del Ig
  if data['High'][-1] <= (data['High'][-2]+ (ign_size*tol)) and data['Low'][-1] >= (data['Low'][-2] + (ign_size/2)):
    trigger_long.append(igniting_long[i])

for i in range(len(igniting_short)):
  data = pd.DataFrame.copy(yfinance.download(igniting_short[i], period='2d', interval='5m')[['Low', 'High']])
  #Condiciones para Trigger: que el Low de la segunda barra este por arriba del Low Igniting Y que el High segunda esté por abajo de la mitad del Ig
  if data['Low'][-1] >= (data['Low'][-2] - (ign_size*tol)) and data['High'][-1] <= (data['High'][-2] - (ign_size/2)):
    trigger_short.append(igniting_short[i])

[*********************100%***********************]  1 of 1 completed


In [14]:
trigger_long

[]

In [15]:
trigger_short

[]